In [56]:
import numpy as np
import pandas as pd

In [122]:
class Node(object):
    def __init__(self,inbound_nodes=[]):
        
        self.inbound_nodes = inbound_nodes
        
        self.outbound_nodes=[]
        
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        
    def forward(self):
        
        raise NotImplemented
        
## Input Class
class Input(Node):
    def __init__(self):
        
        Node.__init__(self)
    
    def forward(self,value=None):
        
        if value is not None:
            self.value = value        

## Topological class
def topological_sort(feed_dict):
    
    input_nodes = [n for n in feed_dict.keys()]
    
    G={}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        
        if n not in G:
            G[n] ={'in': set(),'out': set()}
        for m in n.outbound_nodes:
            
            if m not in G:
                G[m] ={'in': set(),'out': set()}
            
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
    L=[]
    S=set(input_nodes)
    while len(S) > 0:
        n=S.pop()
        
        if isinstance(n,Input):
            n.value = feed_dict[n]
            
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) ==0:
                S.add(m)
    return L
            
    
    
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value


class Add(Node):
    def __init__(self, x, y):
        # You could access `x` and `y` in forward with
        # self.inbound_nodes[0] (`x`) and self.inbound_nodes[1] (`y`)
        Node.__init__(self, [x, y])

    def forward(self):
        
        
        x_value = self.inbound_nodes[0].value
        y_value = self.inbound_nodes[1].value
        self.value = x_value + y_value
        self.value = self.value+ y_value
        

class Multiple(Node):
    # You may need to change this...
    def __init__(self, *inputs):
        Node.__init__(self, inputs)

    def forward(self):
        
        
        self.value=0
        
        for i in self.inbound_nodes:
            self.value =self.value + i.value
            
class Linear(Node):
    def __init__(self, X, W, bias):
        Node.__init__(self, [inputs, weights, bias])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output.

        Your code goes here!
        
        m = self.inbound_nodes[0].value
        n=self.inbound_nodes[1].value
        self.value =0
        
        for i in range(len(self.inbound_nodes[0].value)):
            self.value += m[i] * n[i]
            
        
        self.value+=self.inbound_nodes[2].value

        bias = self.inbound_nodes[2].value
        
        self.value =np.dot(self.inbound_nodes[0].value,self.inbound_nodes[1].value) + bias.T"""
        
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b

        

class Sigmoid(Node):
    """
    You need to fix the `_sigmoid` and `forward` methods.
    """
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        
        """
        This method is separate from `forward` because it
        will be used later with `backward` as well.

        `x`: A numpy array-like object.

        Return the result of the sigmoid function.

        Your code here!
        """
        return 1./(1.+ np.exp(-x))
    def forward(self):
        """
        Set the value of this node to the result of the
        sigmoid function, `_sigmoid`.

        Your code here!
        """
        # This is a dummy value to prevent numpy errors
        # if you test without changing this method.
        input_node = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_node)
     
    
    
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        # TODO: your code here
        diff  =y-a
        self.value =np.mean(diff **2)



## Example for topological

In [113]:
from collections import defaultdict

class Graph:
    def __init__(self,vertices):
        self.graph = defaultdict(list)
        self.V = vertices
    def addEdge(self,u,v):
        self.graph[u].append(v)
    def topologicalSortUtil(self,v,visited,stack):
        visited[v]=True
        for i in self.graph[v]:
            if visited[i] == False:
                self.topologicalSortUtil(i,visited,stack)
        stack.insert(0,v)
        
    def topological(self):
        visited=[False]*self.V
        stack=[]
        
        for i in range(self.V):
            if visited[i]==False:
                
                self.topologicalSortUtil(i,visited,stack)
        print(stack)
        

In [114]:
g=Graph(6)

In [115]:
g.addEdge(5,2)
g.addEdge(5,0)
g.addEdge(4,0)
g.addEdge(4,1)
g.addEdge(2,3)

In [116]:
g.topological()

[5, 4, 2, 3, 1, 0]


In [117]:
x, y = Input(), Input()

f = Add(x, y)


feed_dict = {x: 10, y: 5}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort set the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("({} + {}) + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y],feed_dict[y], output))


(10 + 5) + 5 = 20 (according to miniflow)


In [118]:
x, y, z,k = Input(), Input(), Input(),Input()

f = Multiple(x, y, z,k)

feed_dict = {x: 4, y: 5, z: 10,k:25}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

# should output 19
print("{} + {} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], feed_dict[z], output))


4 + 5 + 10 = 44 (according to miniflow)


In [119]:
inputs, weights, bias = Input(), Input(), Input()

f = Linear(inputs, weights, bias)

feed_dict = {
    inputs: [6, 14, 3 , 1],
    weights: [0.5, 0.25, 1.4,1],
    bias: 2
}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

print(output) # should be 12.7 with this example

13.7


In [120]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)


[ 8.7  6.7]


In [121]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)


[ 0.99983344  0.9987706 ]


In [123]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
output = forward_pass(cost,graph)

"""
Expected output

23.4166666667
"""
print(output)


23.4166666667
